In [ ]:
%pip install qumat[qdp]

In [ ]:
"""
QDP + QML: Full GPU Pipeline (float64)
CPU → GPU (QDP batch encode) → GPU (real projection) → GPU (QML training)
"""

import torch
import torch.nn as nn
import torch.optim as optim
from qumat.qdp import QdpEngine

# ─────────────────────────────────────────────
# 1. Setup
# ─────────────────────────────────────────────
DEVICE_ID = 0
TORCH_DEVICE = torch.device("cuda", DEVICE_ID)
NUM_QUBITS = 2
EPOCHS = 60
LR = 0.01

engine = QdpEngine(DEVICE_ID)

# ─────────────────────────────────────────────
# 2. Raw Data on CPU — float64
# ─────────────────────────────────────────────
raw = torch.tensor([
    [0.5, 0.5, 0.5, 0.5],
    [0.7, 0.1, 0.5, 0.3],
    [0.1, 0.8, 0.4, 0.4],
    [0.6, 0.2, 0.6, 0.4],
], dtype=torch.float64)   # ← float64

labels = torch.tensor([0, 1, 0, 1], dtype=torch.float64, device=TORCH_DEVICE)

# ─────────────────────────────────────────────
# 3. CPU → GPU: QDP Batch Encode
# ─────────────────────────────────────────────
print("CPU → GPU: Batch encoding with QDP...")
cuda_batch = raw.cuda()

qtensor = engine.encode(cuda_batch, num_qubits=NUM_QUBITS, encoding_method="amplitude")

# DLPack → complex128 CUDA tensor (two float64s per element)
X_complex = torch.from_dlpack(qtensor)
print(f"Raw encoded: shape={X_complex.shape}, dtype={X_complex.dtype}, device={X_complex.device}")

# Concatenate real + imag → float64 [N, 8], stays on GPU
X_quantum = torch.cat([X_complex.real, X_complex.imag], dim=-1).double()
print(f"Real features: shape={X_quantum.shape}, dtype={X_quantum.dtype}, device={X_quantum.device}")

# ─────────────────────────────────────────────
# 4. QML Model on GPU — double precision
# ─────────────────────────────────────────────
class VariationalLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.theta = nn.Parameter(torch.randn(dim, dtype=torch.float64))

    def forward(self, x):
        return x * torch.cos(self.theta) + torch.roll(x, 1, dims=-1) * torch.sin(self.theta)

class QMLClassifier(nn.Module):
    def __init__(self, num_qubits):
        super().__init__()
        dim = 2 * (2 ** num_qubits)   # real + imag
        self.layer1 = VariationalLayer(dim)
        self.layer2 = VariationalLayer(dim)
        self.readout = nn.Linear(dim, 1, dtype=torch.float64)

    def forward(self, x):
        x = torch.tanh(self.layer1(x))
        x = self.layer2(x)
        return torch.sigmoid(self.readout(x)).squeeze(-1)

model = QMLClassifier(NUM_QUBITS).to(TORCH_DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.BCELoss()

# ─────────────────────────────────────────────
# 5. GPU Training
# ─────────────────────────────────────────────
print("\nGPU → Training QML model...")
for epoch in range(1, EPOCHS + 1):
    model.train()
    optimizer.zero_grad()
    preds = model(X_quantum)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        with torch.no_grad():
            acc = ((preds > 0.5).double() == labels).double().mean().item()
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.6f} | Accuracy: {acc:.2f}")

# ─────────────────────────────────────────────
# 6. Inference
# ─────────────────────────────────────────────
model.eval()
with torch.no_grad():
    predicted = (model(X_quantum) > 0.5).int()

print("\n─── Results ───")
for i, (pred, true) in enumerate(zip(predicted.cpu().tolist(), labels.int().cpu().tolist())):
    print(f"Sample {i}: Predicted={pred}  True={true}  {'✓' if pred == true else '✗'}")